In [1]:
import pandas as pd
ds=pd.DataFrame(pd.read_csv('weatherid3.csv'))
ds

,Outlook,Temperature,Humidity,Wind,Play Tennis
0,Sunny,Hot,High,Weak,No
1,Sunny,Hot,High,Strong,No
2,Overcast,Hot,High,Weak,Yes
3,Rain,Mild,High,Weak,Yes
4,Rain,Cool,Normal,Weak,Yes
5,Rain,Cool,Normal,Strong,No
6,Overcast,Cool,Normal,Strong,Yes
7,Sunny,Mild,High,Weak,No
8,Sunny,Cool,Normal,Weak,Yes
9,Rain,Mild,Normal,Weak,Yes


In [2]:
def entropy(probs):
    import math
    return sum(-prob*math.log(prob,2) for prob in probs)

In [6]:
def entropy_of_list(a_list):
    from collections import Counter
    cnt=Counter(x for x in a_list)
    print('no and yes instances:',a_list.name,cnt)
    n_ins=len(a_list)*1.0
    prob=[x/n_ins for x in cnt.values()]
    return entropy(prob)    

In [7]:
print('playtennis dataset',ds['Play Tennis'])
t_e=entropy_of_list(ds['Play Tennis'])
print('total entropy:',t_e)

playtennis dataset 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: Play Tennis, dtype: object
no and yes instances: Play Tennis Counter({'Yes': 9, 'No': 5})
total entropy: 0.9402859586706309


In [13]:
def infog(ds,san,tan,t=0):
    print('info gain for',san)
    sds=ds.groupby(san)
    for name,group in sds:
        print(name)
        print(group)
    nobs=len(ds.index)*1.0
    dae=sds.agg({tan:[entropy_of_list,lambda x:len(x)/nobs]})[tan]
    dae.columns=['entropy','propob']
    new_ent=sum(dae['entropy']*dae['propob'])
    old_ent=entropy_of_list(ds[tan])
    return old_ent-new_ent

In [14]:
print('Info gain for outlook'+str(infog(ds,'Outlook','Play Tennis')),"\n")
print('Info gain for humidity'+str(infog(ds,'Humidity','Play Tennis')),"\n")
print('Info gain for wind'+str(infog(ds,'Wind','Play Tennis')),"\n")
print('Info gain for temperature'+str(infog(ds,'Temperature','Play Tennis')),"\n")

info gain for Outlook
Overcast
     Outlook Temperature Humidity    Wind Play Tennis
2   Overcast         Hot     High    Weak         Yes
6   Overcast        Cool   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
Rain
   Outlook Temperature Humidity    Wind Play Tennis
3     Rain        Mild     High    Weak         Yes
4     Rain        Cool   Normal    Weak         Yes
5     Rain        Cool   Normal  Strong          No
9     Rain        Mild   Normal    Weak         Yes
13    Rain        Mild     High  Strong          No
Sunny
   Outlook Temperature Humidity    Wind Play Tennis
0    Sunny         Hot     High    Weak          No
1    Sunny         Hot     High  Strong          No
7    Sunny        Mild     High    Weak          No
8    Sunny        Cool   Normal    Weak         Yes
10   Sunny        Mild   Normal  Strong         Yes
no and yes instances: Play Tennis Counter({'Yes': 4})
no and yes

In [23]:
def id3(ds,tan,an,dc=None):
    from collections import Counter
    cnt=Counter(x for x in ds[tan])
    if len(cnt)==1:
        return next(iter(cnt))
    elif ds.empty or (not an):
        return dc
    else:
        dc=max(cnt.keys())
        gainx=[infog(ds,a,tan) for a in an]
        max_index=gainx.index(max(gainx))
        best_attr=an[max_index]
        tree={best_attr:{}}
        remaining_attr=[i for i in an if i!=best_attr]
        for aval,subd in ds.groupby(best_attr):
            subtree=id3(subd,tan,remaining_attr,dc)
            tree[best_attr][aval]=subtree
        return tree

In [25]:
an=list(ds.columns)
print(an)
an.remove('Play Tennis')
print(an)
tree=id3(ds,'Play Tennis',an)
from pprint import pprint
pprint(tree)
      

['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play Tennis']
['Outlook', 'Temperature', 'Humidity', 'Wind']
info gain for Outlook
Overcast
     Outlook Temperature Humidity    Wind Play Tennis
2   Overcast         Hot     High    Weak         Yes
6   Overcast        Cool   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
Rain
   Outlook Temperature Humidity    Wind Play Tennis
3     Rain        Mild     High    Weak         Yes
4     Rain        Cool   Normal    Weak         Yes
5     Rain        Cool   Normal  Strong          No
9     Rain        Mild   Normal    Weak         Yes
13    Rain        Mild     High  Strong          No
Sunny
   Outlook Temperature Humidity    Wind Play Tennis
0    Sunny         Hot     High    Weak          No
1    Sunny         Hot     High  Strong          No
7    Sunny        Mild     High    Weak          No
8    Sunny        Cool   Normal    Weak         Yes
10   Su